In [177]:
import pandas as pd

In [178]:
df = pd.read_excel("climate_survey-2018.xlsx")

In [179]:
df = df.dropna()

In [180]:
print(df.columns)

Index(['id', 'area', 'sex', 'age', 'edu', 'job', 'marry', 'child', 'income',
       'pol', 'concern', 'satis1', 'satis2', 'satis3', 'satis4', 'satis5',
       'satis6', 'satis7', 'satis8', 'satis_gen', 'satis_mean1', 'satis_mean2',
       'aware1', 'aware2', 'aware3', 'aware4', 'aware5', 'aware6', 'aware7',
       'aware8', 'aware9', 'aware10', 'aware11', 'aware12', 'aware13',
       'aware14', 'aware15', 'aware16', 'aware17', 'aware_tot', 'occur',
       'risk', 'risk_me', 'eco_at', 'prac1', 'prac2', 'prac3', 'prac4',
       'prac5', 'prac6', 'prac7', 'prac8', 'prac9', 'prac_tot'],
      dtype='object')


In [181]:
X_cols = ['concern', 'satis_mean2', 'aware_tot', 
        'occur', 'risk', 'risk_me', 'eco_at']
Y_cols = ['prac_tot']
C_cols = ['sex', 'age', 'area', 'edu', 'job', 'marry', 'child', 'income', 'pol']

In [182]:
data = df[X_cols + Y_cols + C_cols]

In [183]:
# center_cols = ['concern', 'satis_mean2', 'aware_tot', 'occur', 'risk', 'risk_me', 'eco_at', 
#                'prac_tot', 
#                'age', 'edu', 'income', 'pol']
center_cols = ['concern', 'satis_mean2', 'aware_tot', 'occur', 'risk', 'risk_me', 'eco_at', 
               'prac_tot', 
               'age', 'edu', 'income', 'pol']
data_centered = data.copy()

In [184]:
data_centered[center_cols] = data_centered[center_cols].apply(lambda x: x - x.mean())
data_centered.head()

,concern,satis_mean2,aware_tot,occur,risk,risk_me,eco_at,prac_tot,sex,age,area,edu,job,marry,child,income,pol
0,-0.916182,0.078626,-0.789258,-0.080055,0.097503,-0.277455,-0.155662,3.631543,1,-20.112213,8,0.107424,11,1,0,1.913787,-0.857338
1,0.083818,0.303626,5.210742,0.919945,1.097503,0.722545,0.844338,-3.368457,0,-4.112213,13,0.107424,6,2,1,-3.086213,0.142662
2,0.083818,-0.821374,-2.789258,-0.080055,1.097503,0.722545,0.844338,6.631543,0,7.887787,8,0.107424,7,2,1,-1.086213,0.142662
3,0.083818,-0.483874,-1.789258,-0.080055,-0.902497,-1.277455,-1.155662,2.631543,1,-1.112213,15,0.107424,9,2,1,-1.086213,0.142662
4,0.083818,-0.146374,1.210742,-0.080055,-0.902497,-0.277455,-1.155662,-2.368457,1,-7.112213,10,0.107424,6,2,1,1.913787,0.142662


In [185]:
print(data_centered['pol'].isnull().sum())

0


In [186]:
print(data.columns)

Index(['concern', 'satis_mean2', 'aware_tot', 'occur', 'risk', 'risk_me',
       'eco_at', 'prac_tot', 'sex', 'age', 'area', 'edu', 'job', 'marry',
       'child', 'income', 'pol'],
      dtype='object')


In [187]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## 다중회귀분석

### 모든 독립 변인

In [188]:
Y = data['prac_tot']
X = data[X_cols]

X = sm.add_constant(X)

# linear regression
model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               prac_tot   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     106.3
Date:                Fri, 01 Nov 2024   Prob (F-statistic):          5.06e-139
Time:                        00:21:57   Log-Likelihood:                -6213.7
No. Observations:                2923   AIC:                         1.244e+04
Df Residuals:                    2915   BIC:                         1.249e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          12.2571      0.399     30.758      

/Users/min/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [189]:
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

       feature         VIF
0        const  112.595648
1      concern    1.268694
2  satis_mean2    1.126109
3    aware_tot    1.170063
4        occur    1.084852
5         risk    1.592408
6      risk_me    1.546031
7       eco_at    1.185854


#### 평균 중심화

In [190]:
Y = data_centered['prac_tot']
X_centered = data_centered[X_cols]

X_centered = sm.add_constant(X_centered)

# linear regression
model_centered = sm.OLS(Y, X_centered).fit()
print(model_centered.summary())

                            OLS Regression Results                            
Dep. Variable:               prac_tot   R-squared:                       0.203
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     106.3
Date:                Fri, 01 Nov 2024   Prob (F-statistic):          5.06e-139
Time:                        00:21:57   Log-Likelihood:                -6213.7
No. Observations:                2923   AIC:                         1.244e+04
Df Residuals:                    2915   BIC:                         1.249e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        2.151e-16      0.038   5.73e-15      

In [191]:
vif_data_centered = pd.DataFrame()
vif_data_centered["feature"] = X_centered.columns
vif_data_centered["VIF"] = [variance_inflation_factor(X_centered.values, i) for i in range(X_centered.shape[1])]

print(vif_data_centered)

       feature       VIF
0        const  1.000000
1      concern  1.268694
2  satis_mean2  1.126109
3    aware_tot  1.170063
4        occur  1.084852
5         risk  1.592408
6      risk_me  1.546031
7       eco_at  1.185854


#### 통제 변인 다중 회귀

In [192]:
C = data_centered[C_cols]
C = sm.add_constant(C)

# linear regression
modelC = sm.OLS(Y, C).fit()
print(modelC.summary())

                            OLS Regression Results                            
Dep. Variable:               prac_tot   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     13.81
Date:                Fri, 01 Nov 2024   Prob (F-statistic):           5.45e-22
Time:                        00:21:58   Log-Likelihood:                -6485.0
No. Observations:                2923   AIC:                         1.299e+04
Df Residuals:                    2913   BIC:                         1.305e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4096      0.230     -1.778      0.0

In [193]:
vif_data_c = pd.DataFrame()
vif_data_c["feature"] = C.columns
vif_data_c["VIF"] = [variance_inflation_factor(C.values, i) for i in range(C.shape[1])]

print(vif_data_c)

  feature        VIF
0   const  31.230315
1     sex   1.088269
2     age   1.963790
3    area   1.020230
4     edu   1.101365
5     job   1.085712
6   marry   2.909372
7   child   3.152143
8  income   1.084043
9     pol   1.045902


#### 통제 변인 추가

In [194]:
C_cols = ['sex', 'age']

In [195]:
# 성별 & 나이 통제
XC = data_centered[X_cols + C_cols]
XC = sm.add_constant(XC)

# linear regression
model_addC = sm.OLS(Y, XC).fit()
print(model_addC.summary())

                            OLS Regression Results                            
Dep. Variable:               prac_tot   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     86.21
Date:                Fri, 01 Nov 2024   Prob (F-statistic):          1.86e-142
Time:                        00:21:58   Log-Likelihood:                -6201.0
No. Observations:                2923   AIC:                         1.242e+04
Df Residuals:                    2913   BIC:                         1.248e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.1804      0.052      3.455      

In [196]:
vif_data_c = pd.DataFrame()
vif_data_c["feature"] = XC.columns
vif_data_c["VIF"] = [variance_inflation_factor(XC.values, i) for i in range(XC.shape[1])]

print(vif_data_c)

       feature       VIF
0        const  1.947428
1      concern  1.279274
2  satis_mean2  1.137093
3    aware_tot  1.210581
4        occur  1.085670
5         risk  1.593351
6      risk_me  1.565593
7       eco_at  1.207734
8          sex  1.030666
9          age  1.095550


##### 통제 변인으로 지역 추가

In [197]:
data_d = pd.get_dummies(data_centered, columns=['area'], drop_first=True)
data_d.head()

,concern,satis_mean2,aware_tot,occur,risk,risk_me,eco_at,prac_tot,sex,age,...,area_8,area_9,area_10,area_11,area_12,area_13,area_14,area_15,area_16,area_17
0,-0.916182,0.078626,-0.789258,-0.080055,0.097503,-0.277455,-0.155662,3.631543,1,-20.112213,...,1,0,0,0,0,0,0,0,0,0
1,0.083818,0.303626,5.210742,0.919945,1.097503,0.722545,0.844338,-3.368457,0,-4.112213,...,0,0,0,0,0,1,0,0,0,0
2,0.083818,-0.821374,-2.789258,-0.080055,1.097503,0.722545,0.844338,6.631543,0,7.887787,...,1,0,0,0,0,0,0,0,0,0
3,0.083818,-0.483874,-1.789258,-0.080055,-0.902497,-1.277455,-1.155662,2.631543,1,-1.112213,...,0,0,0,0,0,0,0,1,0,0
4,0.083818,-0.146374,1.210742,-0.080055,-0.902497,-0.277455,-1.155662,-2.368457,1,-7.112213,...,0,0,1,0,0,0,0,0,0,0


In [198]:
print(data_d.columns)

Index(['concern', 'satis_mean2', 'aware_tot', 'occur', 'risk', 'risk_me',
       'eco_at', 'prac_tot', 'sex', 'age', 'edu', 'job', 'marry', 'child',
       'income', 'pol', 'area_2', 'area_3', 'area_4', 'area_5', 'area_6',
       'area_7', 'area_8', 'area_9', 'area_10', 'area_11', 'area_12',
       'area_13', 'area_14', 'area_15', 'area_16', 'area_17'],
      dtype='object')


In [199]:
area_cols = ['area_2', 'area_3', 'area_4', 'area_5', 
        'area_6', 'area_7', 'area_8', 'area_9', 'area_10', 
        'area_11', 'area_12', 'area_13', 'area_14', 
        'area_15', 'area_16', 'area_17']
C_cols = C_cols + area_cols

In [200]:
# 지역
XA = data_d[C_cols]
XA = sm.add_constant(XA)

# 모든 행 출력
pd.set_option('display.max_rows', None)  

# linear regression
model_A = sm.OLS(Y, XA).fit()
print(model_A.summary())

                            OLS Regression Results                            
Dep. Variable:               prac_tot   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     6.783
Date:                Fri, 01 Nov 2024   Prob (F-statistic):           4.33e-17
Time:                        00:21:58   Log-Likelihood:                -6485.9
No. Observations:                2923   AIC:                         1.301e+04
Df Residuals:                    2904   BIC:                         1.312e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1940      0.102     -1.907      0.0

In [201]:
# 성별 및 지역 통제
XC = data_d[X_cols + C_cols]
XC = sm.add_constant(XC)

# linear regression
model_addC = sm.OLS(Y, XC).fit()
print(model_addC.summary())

                            OLS Regression Results                            
Dep. Variable:               prac_tot   R-squared:                       0.223
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     33.31
Date:                Fri, 01 Nov 2024   Prob (F-statistic):          6.83e-139
Time:                        00:21:58   Log-Likelihood:                -6176.9
No. Observations:                2923   AIC:                         1.241e+04
Df Residuals:                    2897   BIC:                         1.256e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.1347      0.092     -1.465      